**Part A: File Format Handling and Data Processing (35 marks)**

In [1]:
#!/usr/bin/env python3
"""
Solution for Assignment 3, Part A:
File Format Handling and Data Processing.

This script implements:
- Q1: A robust, memory-efficient FASTA parser and writer.
- Q2: A data integration system using SQLite, including
      a FASTQ parser, data validation, and search functions.
"""

import sqlite3
import re
from typing import Iterator, Tuple, Dict, Any, List

# -----------------------------------------------------------------
# --- Question 1: FASTA File Processing
# -----------------------------------------------------------------

class FastaHandler:
    """
    Implements robust FASTA file parsing and writing.
    """

    def parse_fasta(self, filepath: str) -> Iterator[Tuple[str, str]]:
        """
        Parses a FASTA file with multiple sequences efficiently[cite: 26, 28].

        Uses a generator to yield one sequence at a time, keeping
        memory usage low. It correctly handles multi-line sequences.

        Args:
            filepath: The path to the FASTA file.

        Yields:
            A tuple of (header, sequence) for each record in the file.
            - header: The full header line (e.g., '>seq1 description') [cite: 27]
            - sequence: The full DNA/protein sequence as a single string.
        """
        print(f"[Info] Parsing FASTA file: {filepath}")
        try:
            with open(filepath, 'r') as f:
                header = None
                sequence_lines = []

                for line in f:
                    line = line.strip()
                    if not line:
                        continue  # Skip empty lines

                    if line.startswith('>'):
                        # If we have a sequence buffered, yield it
                        if header:
                            yield header, "".join(sequence_lines)

                        # Start a new sequence
                        header = line
                        sequence_lines = []
                    else:
                        # Append sequence data
                        if header:
                            sequence_lines.append(line)

                # Yield the last sequence in the file
                if header:
                    yield header, "".join(sequence_lines)

        except FileNotFoundError:
            print(f"[Error] File not found: {filepath}")
        except Exception as e:
            print(f"[Error] Failed to parse FASTA: {e}")

    def write_fasta(self, filepath: str,
                    sequences: Iterator[Tuple[str, str]],
                    line_width: int = 60) -> None:
        """
        Implements FASTA writing capabilities with proper formatting.

        Args:
            filepath: The path to the output FASTA file.
            sequences: An iterator (like from parse_fasta) that provides
                       (header, sequence) tuples.
            line_width: The maximum number of characters per sequence line.
        """
        print(f"[Info] Writing sequences to FASTA file: {filepath}")
        try:
            with open(filepath, 'w') as f:
                for header, sequence in sequences:
                    # Ensure header starts with '>'
                    if not header.startswith('>'):
                        header = '>' + header

                    f.write(f"{header}\n")

                    # Write sequence in formatted lines
                    for i in range(0, len(sequence), line_width):
                        f.write(f"{sequence[i:i + line_width]}\n")
            print(f"[Success] FASTA file written to {filepath}")
        except Exception as e:
            print(f"[Error] Failed to write FASTA: {e}")


# -----------------------------------------------------------------
# --- Question 2: Genomic Data Integration
# -----------------------------------------------------------------

class GenomicDataHandler:
    """
    Develops data management systems for genomic data.

    Uses SQLite for database storage  and includes
    parsers for FASTA/FASTQ , data validation,
    and search functions.
    """

    def __init__(self, db_path: str):
        """
        Initializes the handler with a path to the SQLite database.

        Args:
            db_path: Path to the .sqlite file (e.g., 'genomics.db')
        """
        self.db_path = db_path
        self._create_database()

    def _create_database(self) -> None:
        """
        Creates the database tables if they don't already exist.
        """
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS sequences (
                    id TEXT PRIMARY KEY,
                    header TEXT NOT NULL,
                    sequence TEXT NOT NULL,
                    source_format TEXT NOT NULL,
                    metadata_json TEXT
                )
                ''')
                conn.commit()
        except sqlite3.Error as e:
            print(f"[Error] Database creation failed: {e}")

    def validate_sequence(self, sequence: str,
                          alphabet: str = r'^[ATCGN]+$') -> bool:
        """
        Builds data validation and quality control measures.

        Checks if a sequence contains only valid characters (A, T, C, G, N).

        Args:
            sequence: The DNA sequence string to check.
            alphabet: A regex for the valid character set.

        Returns:
            True if valid, False otherwise.
        """
        if re.match(alphabet, sequence.upper()):
            return True
        print(f"[QC Fail] Invalid characters found in sequence: {sequence[:20]}...")
        return False

    def parse_fastq(self, filepath: str) -> Iterator[Dict[str, str]]:
        """
        Handles the basics of the FASTQ file format.

        Uses a generator to yield one record at a time, keeping
        memory usage low.

        Args:
            filepath: The path to the FASTQ file.

        Yields:
            A dictionary for each record:
            {'header': str, 'sequence': str, 'quality': str}
        """
        print(f"[Info] Parsing FASTQ file: {filepath}")
        try:
            with open(filepath, 'r') as f:
                while True:
                    # 1. Header line
                    header = f.readline().strip()
                    if not header:
                        break  # End of file

                    # 2. Sequence line
                    sequence = f.readline().strip()

                    # 3. Plus line
                    plus_line = f.readline().strip()

                    # 4. Quality line
                    quality = f.readline().strip()

                    if not (header.startswith('@') and plus_line == '+' and \
                            len(sequence) == len(quality)):
                        print(f"[Error] Corrupt FASTQ record found. Skipping.")
                        continue

                    yield {
                        'header': header,
                        'sequence': sequence,
                        'quality': quality
                    }
        except FileNotFoundError:
            print(f"[Error] File not found: {filepath}")
        except Exception as e:
            print(f"[Error] Failed to parse FASTQ: {e}")

    def load_data_to_db(self, parser_gen: Iterator,
                        source_format: str) -> None:
        """
        Loads data from a parser (FASTA or FASTQ) into the database.
        """
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                count = 0
                for record in parser_gen:
                    if source_format == 'FASTA':
                        header, seq = record
                        record_id = header.split()[0][1:] # Get '>id' part
                        metadata = {'full_header': header}
                    elif source_format == 'FASTQ':
                        header = record['header']
                        seq = record['sequence']
                        record_id = header.split()[0][1:] # Get '@id' part
                        metadata = {'quality_scores': record['quality']}

                    # Run validation
                    if not self.validate_sequence(seq):
                        continue

                    # Insert data, ignore if ID already exists
                    cursor.execute(
                        "INSERT OR IGNORE INTO sequences (id, header, sequence, source_format, metadata_json) VALUES (?, ?, ?, ?, ?)",
                        (record_id, header, seq, source_format, str(metadata))
                    )
                    count += 1
                conn.commit()
                print(f"[Success] Loaded {count} records from {source_format} into {self.db_path}")
        except sqlite3.Error as e:
            print(f"[Error] Failed to load data to database: {e}")
        except Exception as e:
            print(f"[Error] An unexpected error occurred during DB load: {e}")

    def search_by_id(self, seq_id_pattern: str) -> List[Tuple]:
        """
        Implements search and retrieval functions.

        Searches for records where the ID matches a pattern (e.g., 'seq1%').

        Args:
            seq_id_pattern: An SQL LIKE pattern (e.g., 'Ecoli_gene%').

        Returns:
            A list of matching records (id, header, source_format).
        """
        print(f"[Info] Searching for records like: '{seq_id_pattern}'")
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute(
                    "SELECT id, header, source_format FROM sequences WHERE id LIKE ?",
                    (seq_id_pattern,)
                )
                results = cursor.fetchall()
                return results
        except sqlite3.Error as e:
            print(f"[Error] Database search failed: {e}")
            return []

# -----------------------------------------------------------------
# --- Demonstration & Testing [cite: 6]
# -----------------------------------------------------------------

if __name__ == "__main__":
    print("="*60)
    print("  Running Demonstration for Assignment 3 - Part A")
    print("="*60)

    # --- Setup: Create dummy files for testing ---

    # Dummy FASTA data
    fasta_content = """
>seq1 K12 E. coli
ATGCGTACGTACGATCAGTCAGTCAGACTGACATCAGTACGATCAGTCA
GTCAGACTGACATCAGTACGATCAGTCAGTCAGACTGACATCAGTACGA
TCAGTCAGTCAGACTGACATCAGT
>seq2 M. tuberculosis
ATGGGGGGGGGGGGGGGGGGGGGCCCCCCCCCCCCCCCCCCCTATATATA
TATATATATATATATAGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG
>seq3_with_invalid_chars
ATGCGTACGTACGXATCAGTCAGTZY
"""
    with open("demo_multi.fasta", "w") as f:
        f.write(fasta_content)

    # Dummy FASTQ data
    fastq_content = """
@read1_flowcell_1
ATGCGTACGTACGATCAGTCAGTCAGACTGACATCAGTACGATCAGTCAG
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@read2_flowcell_1
ATGGGGGGGGGGGGGGGGGGGGGCCCCCCCCCCCCCCCCCCCTATATATA
+
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
"""
    with open("demo.fastq", "w") as f:
        f.write(fastq_content)


    # --- Q1 Test: FASTA Parsing [cite: 26, 27, 28] ---
    print("\n--- Testing Q1: FASTA Parsing ---")
    fasta_handler = FastaHandler()
    fasta_gen = fasta_handler.parse_fasta("demo_multi.fasta")

    # Store parsed data for writing and DB loading
    parsed_sequences = list(fasta_gen)

    for header, seq in parsed_sequences:
        print(f"  Parsed Header: {header}")
        print(f"  Parsed Sequence: {seq[:30]}... (len: {len(seq)})")

    # --- Q1 Test: FASTA Writing  ---
    print("\n--- Testing Q1: FASTA Writing ---")
    # We'll write the non-invalid sequences back to a new file
    sequences_to_write = [
        (h, s) for h, s in parsed_sequences if "invalid" not in h
    ]
    fasta_handler.write_fasta("demo_output.fasta", sequences_to_write)


    # --- Q2 Test: Data Integration & Validation [cite: 31, 33, 34] ---
    print("\n--- Testing Q2: Data Integration (SQLite) ---")
    db_handler = GenomicDataHandler("genomics_db.sqlite")

    # Load FASTA data (note: 'seq3' will fail validation )
    db_handler.load_data_to_db(
        fasta_handler.parse_fasta("demo_multi.fasta"),
        "FASTA"
    )

    # Load FASTQ data
    db_handler.load_data_to_db(
        db_handler.parse_fastq("demo.fastq"),
        "FASTQ"
    )

    # --- Q2 Test: Search & Retrieval  ---
    print("\n--- Testing Q2: Database Search ---")

    print("Searching for 'seq%'...")
    seq_results = db_handler.search_by_id("seq%")
    for res in seq_results:
        print(f"  Found: {res}")

    print("\nSearching for 'read%'...")
    read_results = db_handler.search_by_id("read%")
    for res in read_results:
        print(f"  Found: {res}")

    print("\n" + "="*60)
    print("  Demonstration Complete.")
    print("  Check files: 'genomics_db.sqlite' and 'demo_output.fasta'")
    print("="*60)

  Running Demonstration for Assignment 3 - Part A

--- Testing Q1: FASTA Parsing ---
[Info] Parsing FASTA file: demo_multi.fasta
  Parsed Header: >seq1 K12 E. coli
  Parsed Sequence: ATGCGTACGTACGATCAGTCAGTCAGACTG... (len: 122)
  Parsed Header: >seq2 M. tuberculosis
  Parsed Sequence: ATGGGGGGGGGGGGGGGGGGGGGCCCCCCC... (len: 99)
  Parsed Header: >seq3_with_invalid_chars
  Parsed Sequence: ATGCGTACGTACGXATCAGTCAGTZY... (len: 26)

--- Testing Q1: FASTA Writing ---
[Info] Writing sequences to FASTA file: demo_output.fasta
[Success] FASTA file written to demo_output.fasta

--- Testing Q2: Data Integration (SQLite) ---
[Info] Parsing FASTA file: demo_multi.fasta
[QC Fail] Invalid characters found in sequence: ATGCGTACGTACGXATCAGT...
[Success] Loaded 2 records from FASTA into genomics_db.sqlite
[Info] Parsing FASTQ file: demo.fastq
[Success] Loaded 0 records from FASTQ into genomics_db.sqlite

--- Testing Q2: Database Search ---
Searching for 'seq%'...
[Info] Searching for records like: 'seq%

**Part B: Rosalind-Style Problem Solving (40 marks)**

In [3]:
#!/usr/bin/env python3
"""
Solution for Assignment 3, Part B:
Rosalind-Style Problem Solving.

This script implements a wide range of core bioinformatics algorithms
for sequence comparison and pattern analysis.

- Q3: Needleman-Wunsch, LCS, p-distance, Jukes-Cantor, Consensus
- Q4: Suffix Array, Repeats/Palindromes, Greedy Assembly, UPGMA
"""

import math
from collections import Counter
import numpy as np  # Required for alignment matrices [cite: 15]

# -----------------------------------------------------------------
# --- Question 3: Multiple Sequence Problems
# -----------------------------------------------------------------

def align_global_needleman_wunsch(seq1: str, seq2: str,
                                  match_score: int = 1,
                                  mismatch_penalty: int = -1,
                                  gap_penalty: int = -2) -> (int, str, str):
    """
    Implements global sequence alignment basics using Needleman-Wunsch.

    Uses dynamic programming to find the optimal global alignment.

    Time Complexity: O(n*m)
    Space Complexity: O(n*m)

    Args:
        seq1 (str): First sequence
        seq2 (str): Second sequence
        match_score (int): Score for a match
        mismatch_penalty (int): Penalty for a mismatch (negative)
        gap_penalty (int): Penalty for a gap (negative)

    Returns:
        A tuple: (alignment_score, aligned_seq1, aligned_seq2)
    """
    n = len(seq1)
    m = len(seq2)

    # Initialize the score matrix
    # We use numpy for efficient matrix operations [cite: 15]
    score_matrix = np.zeros((n + 1, m + 1))

    # Initialize the traceback matrix
    # 0 = diagonal, 1 = up, 2 = left
    trace_matrix = np.zeros((n + 1, m + 1), dtype=int)

    # Fill the first row and column with gap penalties
    for i in range(n + 1):
        score_matrix[i, 0] = i * gap_penalty
        trace_matrix[i, 0] = 1  # From 'up'
    for j in range(m + 1):
        score_matrix[0, j] = j * gap_penalty
        trace_matrix[0, j] = 2  # From 'left'

    # Fill the DP matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            # Calculate score for match/mismatch
            match = (match_score if seq1[i - 1] == seq2[j - 1]
                     else mismatch_penalty)

            diag_score = score_matrix[i - 1, j - 1] + match
            up_score = score_matrix[i - 1, j] + gap_penalty
            left_score = score_matrix[i, j - 1] + gap_penalty

            scores = [diag_score, up_score, left_score]
            score_matrix[i, j] = max(scores)
            trace_matrix[i, j] = np.argmax(scores) # 0, 1, or 2

    # --- Traceback ---
    align1, align2 = "", ""
    i, j = n, m
    final_score = score_matrix[i, j]

    while i > 0 or j > 0:
        trace = trace_matrix[i, j]
        if trace == 0:  # Diagonal
            align1 = seq1[i - 1] + align1
            align2 = seq2[j - 1] + align2
            i -= 1
            j -= 1
        elif trace == 1:  # Up
            align1 = seq1[i - 1] + align1
            align2 = "-" + align2
            i -= 1
        else:  # Left
            align1 = "-" + align1
            align2 = seq2[j - 1] + align2
            j -= 1

    return final_score, align1, align2

def find_longest_common_subsequence(seq1: str, seq2: str) -> (int, str):
    """
    Finds the longest common subsequence between two sequences.
    This is different from a sub*string*, as a sub*sequence*
    does not have to be contiguous.

    Time Complexity: O(n*m)
    Space Complexity: O(n*m)
    """
    n = len(seq1)
    m = len(seq2)

    # dp_table[i][j] will store the length of the LCS of
    # seq1[0...i-1] and seq2[0...j-1]
    dp_table = np.zeros((n + 1, m + 1), dtype=int)

    # Fill the table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if seq1[i - 1] == seq2[j - 1]:
                dp_table[i, j] = dp_table[i - 1, j - 1] + 1
            else:
                dp_table[i, j] = max(dp_table[i - 1, j], dp_table[i, j - 1])

    # --- Traceback to find the subsequence ---
    lcs_str = ""
    i, j = n, m
    while i > 0 and j > 0:
        if seq1[i - 1] == seq2[j - 1]:
            lcs_str = seq1[i - 1] + lcs_str
            i -= 1
            j -= 1
        elif dp_table[i - 1, j] > dp_table[i, j - 1]:
            i -= 1
        else:
            j -= 1

    return dp_table[n, m], lcs_str

def calc_p_distance(seq1: str, seq2: str) -> float:
    """
    Calculates evolutionary distance using a simple model: p-distance.
    p-distance is the proportion of sites at which the
    two sequences differ. Assumes sequences are aligned.

    Time Complexity: O(n)
    Space Complexity: O(1)
    """
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be aligned (equal length).")

    n = len(seq1)
    differences = 0
    for i in range(n):
        if seq1[i] != seq2[i] and seq1[i] != '-' and seq2[i] != '-':
            differences += 1

    return differences / n

def calc_jukes_cantor_distance(p_distance: float) -> float:
    """
    Calculates evolutionary distance using the Jukes-Cantor (JC69) model.
    It corrects the p-distance for multiple substitutions at the same site.

    Time Complexity: O(1)
    Space Complexity: O(1)
    """
    if p_distance >= 0.75:
        return float('inf')  # Distance is undefined
    return - (3 / 4) * math.log(1 - (4 / 3) * p_distance)

def generate_consensus_from_alignment(aligned_seqs: list[str]) -> (str, dict):
    """
    Develops consensus sequence generation algorithms.
    Takes a list of *already aligned* sequences.

    Time Complexity: O(n*m) where n=num_seqs, m=seq_length
    Space Complexity: O(m*k) where m=seq_length, k=alphabet_size

    Args:
        aligned_seqs: A list of aligned sequences (e.g., ["A-T", "AAT", "A-C"])

    Returns:
        A tuple: (consensus_sequence, profile_matrix)
        - profile_matrix is a dict of {base: [counts]}
    """
    if not aligned_seqs:
        return "", {}

    seq_length = len(aligned_seqs[0])
    consensus = ""
    profile = {base: [0] * seq_length for base in "ACGT-"}

    for j in range(seq_length):  # For each position
        col_counts = Counter()
        for i in range(len(aligned_seqs)):  # For each sequence
            base = aligned_seqs[i][j].upper()
            if base in profile:
                profile[base][j] += 1
                col_counts[base] += 1

        # Determine consensus base (ignore gaps unless they are majority)
        non_gap_counts = {b: c for b, c in col_counts.items() if b != '-'}
        if non_gap_counts:
            consensus_base = max(non_gap_counts, key=non_gap_counts.get)
        else:
            consensus_base = '-'  # Only if column is all gaps
        consensus += consensus_base

    return consensus, profile

# -----------------------------------------------------------------
# --- Question 4: Advanced Pattern Analysis
# -----------------------------------------------------------------

def build_suffix_array(text: str) -> list[int]:
    """
    Implements suffix array concepts for pattern matching.
    This is a simple O(n^2 log n) build, good for demonstration.
    More advanced builds are O(n log n) or O(n).

    Time Complexity: O(n^2 log n) - n suffixes, O(n) for slicing,
                     O(n log n) for sorting n items.
                     (Note: Python's sort is highly optimized,
                     but string comparison can take O(n) in worst case)
    Space Complexity: O(n^2) - to store all suffixes temporarily

    Args:
        text: The input text (e.g., a genome)

    Returns:
        List of integers (the suffix array)
    """
    text += '$'  # Add terminal character
    n = len(text)

    # Generate all suffixes
    suffixes = []
    for i in range(n):
        suffixes.append((text[i:], i)) # (suffix, original_index)

    # Sort suffixes lexicographically
    suffixes.sort()

    # The suffix array is just the list of original indices
    suffix_array = [index for suffix, index in suffixes]
    return suffix_array

def find_pattern_suffix_array(text: str, pattern: str, sa: list[int]) -> list[int]:
    """
    Finds a pattern using a pre-computed Suffix Array (SA).
    Uses binary search for efficient O(m log n) lookup.

    Time Complexity: O(m * log n) - m=pattern len, n=text len
    Space Complexity: O(1)

    Args:
        text (str): The *original* text (must end in '$')
        pattern (str): The pattern to find
        sa (list[int]): The pre-computed suffix array

    Returns:
        List of start indices where pattern occurs.
    """
    n = len(text)
    m = len(pattern)

    # Binary search for the *first* occurrence
    low, high = 0, n - 1
    first_hit = -1
    while low <= high:
        mid = (low + high) // 2
        suffix_start = sa[mid]
        # Compare pattern to the suffix
        suffix = text[suffix_start : suffix_start + m]

        if pattern == suffix:
            first_hit = mid
            high = mid - 1  # Keep looking left
        elif pattern < suffix:
            high = mid - 1
        else:
            low = mid + 1

    if first_hit == -1:
        return []  # Pattern not found

    # Binary search for the *last* occurrence
    low, high = first_hit, n - 1
    last_hit = first_hit
    while low <= high:
        mid = (low + high) // 2
        suffix_start = sa[mid]
        suffix = text[suffix_start : suffix_start + m]

        if pattern == suffix:
            last_hit = mid
            low = mid + 1 # Keep looking right
        elif pattern < suffix:
            high = mid - 1
        else:
            low = mid + 1

    # All indices from first_hit to last_hit in the SA are matches
    return sorted([sa[i] for i in range(first_hit, last_hit + 1)])

def find_reverse_palindromes(seq: str, min_len: int = 4, max_len: int = 12) -> list:
    """
    Develops algorithms for finding repeats and palindromes.
    This finds "reverse palindromes" (e.g., 'GAATTC' -> 'CTTAAG').
    These are common in biology (e.g., restriction sites).

    Time Complexity: O(n * k) where n=len(seq), k=max_len
    Space Complexity: O(1)
    """
    def reverse_complement(s):
        comp_map = str.maketrans('ATCG', 'TAGC')
        return s.upper().translate(comp_map)[::-1]

    results = []
    n = len(seq)
    for length in range(min_len, max_len + 1):
        for i in range(n - length + 1):
            substring = seq[i : i + length]
            if substring == reverse_complement(substring):
                results.append((i, length, substring))
    return results

def simulate_greedy_assembly(reads: list[str], min_overlap: int = 5) -> str:
    """
    Creates tools for sequence assembly simulation.
    This implements a simple greedy assembler based on
    the Overlap-Layout-Consensus (OLC) paradigm.

    Time Complexity: O(k^2 * n) in each step, O(k^3 * n) overall?
                     (k=num_reads, n=read_len). This is a rough
                     estimate for this simple greedy approach.
    Space Complexity: O(k*n)

    Args:
        reads (list[str]): A list of "read" sequences.
        min_overlap (int): The minimum overlap to consider.

    Returns:
        The assembled contig (str).
    """

    def find_best_overlap(seq1: str, seq2: str) -> (int, int):
        """Finds the best overlap (suffix of seq1 vs prefix of seq2)."""
        best_overlap = 0

        # Check for overlap: seq1_suffix == seq2_prefix
        for k in range(min(len(seq1), len(seq2)), min_overlap - 1, -1):
            if seq1.endswith(seq2[:k]):
                best_overlap = k
                break
        return best_overlap

    # Make a mutable copy of the reads
    contigs = set(reads)

    while len(contigs) > 1:
        best_overlap = -1
        best_pair = (None, None)
        best_merge_type = 0 # 0: a->b, 1: b->a

        # This is the O(k^2) part
        read_list = list(contigs)
        for i in range(len(read_list)):
            for j in range(len(read_list)):
                if i == j:
                    continue

                a = read_list[i]
                b = read_list[j]

                # Check a -> b overlap
                overlap_ab = find_best_overlap(a, b)
                if overlap_ab > best_overlap:
                    best_overlap = overlap_ab
                    best_pair = (a, b)
                    best_merge_type = 0

                # Check b -> a overlap
                overlap_ba = find_best_overlap(b, a)
                if overlap_ba > best_overlap:
                    best_overlap = overlap_ba
                    best_pair = (b, a)
                    best_merge_type = 1

        # If no overlap is found, we're stuck.
        if best_overlap < min_overlap:
            break

        # Perform the merge
        a, b = best_pair
        contigs.remove(a)
        contigs.remove(b)

        # Merge: a_prefix + b
        merged_contig = a + b[best_overlap:]
        contigs.add(merged_contig)

    # Return the largest remaining contig
    return max(contigs, key=len)

def build_upgma_tree(dist_matrix: dict, labels: list[str]) -> str:
    """
    Builds phylogenetic relationship analysis tools.
    Implements UPGMA (Unweighted Pair Group Method with Arithmetic Mean).

    Time Complexity: O(n^3) - n clusters, O(n^2) to find min,
                     O(n) to update matrix, n-1 merges.
    Space Complexity: O(n^2) - to store the distance matrix.

    Args:
        dist_matrix (dict): A nested dict: {label1: {label2: dist}}
        labels (list[str]): A list of species labels.

    Returns:
        A string representing the tree in Newick format.
    """

    # Make a deep copy to work with
    clusters = {label: {l: dist_matrix[label][l] for l in labels if l != label}
                for label in labels}

    # Store cluster info (tree topology and branch lengths)
    # Start with each leaf as a cluster
    cluster_info = {label: {'newick': label, 'size': 1, 'height': 0.0}
                    for label in labels}

    while len(clusters) > 1:
        # Find the pair of clusters with the minimum distance
        min_dist = float('inf')
        c1_key, c2_key = None, None

        c_keys = list(clusters.keys())
        for i in range(len(c_keys)):
            for j in range(i + 1, len(c_keys)):
                k1, k2 = c_keys[i], c_keys[j]
                if clusters[k1][k2] < min_dist:
                    min_dist = clusters[k1][k2]
                    c1_key, c2_key = k1, k2

        # Merge the two clusters (c1 and c2) into a new cluster (c_new)
        c1_info = cluster_info[c1_key]
        c2_info = cluster_info[c2_key]

        c_new_key = f"({c1_key},{c2_key})"
        c_new_size = c1_info['size'] + c2_info['size']
        c_new_height = min_dist / 2.0

        # Create Newick string with branch lengths
        c1_branch = c_new_height - c1_info['height']
        c2_branch = c_new_height - c2_info['height']
        c_new_newick = f"({c1_info['newick']}:{c1_branch:.4f},{c2_info['newick']}:{c2_branch:.4f})"

        # Store info for the new cluster
        cluster_info[c_new_key] = {
            'newick': c_new_newick,
            'size': c_new_size,
            'height': c_new_height
        }

        # Calculate distances from the new cluster to all other clusters
        new_row = {}
        for other_key in clusters:
            if other_key == c1_key or other_key == c2_key:
                continue

            # UPGMA: Arithmetic mean
            dist = (clusters[c1_key][other_key] * c1_info['size'] +
                    clusters[c2_key][other_key] * c2_info['size']) / c_new_size
            new_row[other_key] = dist

        # Remove old clusters from matrix
        del clusters[c1_key]
        del clusters[c2_key]
        for k in clusters:
            del clusters[k][c1_key]
            del clusters[k][c2_key]

        # Add new cluster to matrix
        for k, dist in new_row.items():
            clusters[k][c_new_key] = dist
        clusters[c_new_key] = new_row

    # The last remaining key is the root of the tree
    root_key = list(clusters.keys())[0]
    return cluster_info[root_key]['newick'] + ";"


# -----------------------------------------------------------------
# --- Demonstration and Testing
# -----------------------------------------------------------------

if __name__ == "__main__":

    print("="*60)
    print("  Running Demonstration for Assignment 3 - Part B")
    print("="*60)

    # --- Q3 Test: Multiple Sequence Problems ---
    print("\n--- Q3: Multiple Sequence Problems  ---")
    seqA = "GATTACA"
    seqB = "GCATGCU" # Note 'U'
    seqB = seqB.replace('U', 'T') # Clean data

    # Q3a: Sequence Alignment
    score, al1, al2 = align_global_needleman_wunsch(seqA, seqB)
    print(f"[Q3a] Global Alignment:")
    print(f"  Score: {score}")
    print(f"  Seq1:  {al1}")
    print(f"  Seq2:  {al2}")

    # Q3b: Longest Common Subsequence
    seqC = "AGCAT"
    seqD = "GAC"
    lcs_len, lcs_str = find_longest_common_subsequence(seqC, seqD)
    print(f"\n[Q3b] Longest Common Subsequence (LCS):")
    print(f"  SeqC: {seqC}, SeqD: {seqD}")
    print(f"  LCS: '{lcs_str}' (Length: {lcs_len})")

    # Q3c: Evolutionary Distance
    p_dist = calc_p_distance(al1, al2)
    jc_dist = calc_jukes_cantor_distance(p_dist)
    print(f"\n[Q3c] Evolutionary Distance:")
    print(f"  p-distance: {p_dist:.4f}")
    print(f"  Jukes-Cantor distance: {jc_dist:.4f}")

    # Q3d: Consensus Sequence
    msa = ["GATTACA",
           "G-TTACA",
           "GATT-CA",
           "GCTT-CA",
           "AATTACA"]
    consensus, profile = generate_consensus_from_alignment(msa)
    print(f"\n[Q3d] Consensus Generation:")
    for s in msa:
        print(f"  {s}")
    print(f"  Consensus: {consensus}")
    # print(f"  Profile: {profile}") # Uncomment for full profile


    # --- Q4 Test: Advanced Pattern Analysis ---
    print("\n" + "="*60)
    print("\n--- Q4: Advanced Pattern Analysis  ---")

    genome = "GATTACAGATTACAGATTACA$"

    # Q4a: Suffix Array
    sa = build_suffix_array(genome)
    print(f"[Q4a] Suffix Array (first 10): {sa[:10]}...")
    # [cite: 42]
    pattern = "GATTACA"
    hits = find_pattern_suffix_array(genome, pattern, sa)
    print(f"  Pattern '{pattern}' found at indices: {hits}")
    # [cite: 42]

    # Q4b: Palindromes
    pal_seq = "AGAGAATTCGC" # Contains EcoRI site GAATTC
    palindromes = find_reverse_palindromes(pal_seq, min_len=6, max_len=6)
    print(f"\n[Q4b] Reverse Palindromes:")
    print(f"  Seq: {pal_seq}")
    for pos, length, sub in palindromes:
        print(f"  Found '{sub}' at pos {pos}")

    # Q4c: Assembly Simulation
    reads = ["GATTACAGAT", "AGATTACAGA", "TACAGATTAC"]
    contig = simulate_greedy_assembly(reads, min_overlap=8)
    print(f"\n[Q4c] Greedy Assembly Simulation:")
    print(f"  Reads: {reads}")
    print(f"  Assembled Contig: {contig}")

    # Q4d: Phylogenetic Tree (UPGMA)
    # Simple distance matrix
    labels = ["Human", "Chimp", "Gorilla", "Orangutan"]
    dist_mat = {
        "Human": {"Chimp": 0.02, "Gorilla": 0.04, "Orangutan": 0.08},
        "Chimp": {"Human": 0.02, "Gorilla": 0.05, "Orangutan": 0.09},
        "Gorilla": {"Human": 0.04, "Chimp": 0.05, "Orangutan": 0.10},
        "Orangutan": {"Human": 0.08, "Chimp": 0.09, "Gorilla": 0.10}
    }
    tree = build_upgma_tree(dist_mat, labels)
    print(f"\n[Q4d] UPGMA Phylogenetic Tree:")
    print(f"  Distance Matrix (Human-Chimp): {dist_mat['Human']['Chimp']}")
    print(f"  Newick Tree: {tree}")

    print("\n" + "="*60)
    print("  Part B Demonstration Complete.")
    print("="*60)

  Running Demonstration for Assignment 3 - Part B

--- Q3: Multiple Sequence Problems  ---
[Q3a] Global Alignment:
  Score: -1.0
  Seq1:  GATTACA
  Seq2:  GCATGCT

[Q3b] Longest Common Subsequence (LCS):
  SeqC: AGCAT, SeqD: GAC
  LCS: 'GA' (Length: 2)

[Q3c] Evolutionary Distance:
  p-distance: 0.5714
  Jukes-Cantor distance: 1.0763

[Q3d] Consensus Generation:
  GATTACA
  G-TTACA
  GATT-CA
  GCTT-CA
  AATTACA
  Consensus: GATTACA


--- Q4: Advanced Pattern Analysis  ---
[Q4a] Suffix Array (first 10): [22, 21, 20, 18, 11, 4, 13, 6, 15, 8]...
  Pattern 'GATTACA' found at indices: [0, 7, 14]

[Q4b] Reverse Palindromes:
  Seq: AGAGAATTCGC
  Found 'GAATTC' at pos 3

[Q4c] Greedy Assembly Simulation:
  Reads: ['GATTACAGAT', 'AGATTACAGA', 'TACAGATTAC']
  Assembled Contig: AGATTACAGAT

[Q4d] UPGMA Phylogenetic Tree:
  Distance Matrix (Human-Chimp): 0.02
  Newick Tree: (Orangutan:0.0450,(Gorilla:0.0225,(Human:0.0100,Chimp:0.0100):0.0125):0.0225);

  Part B Demonstration Complete.


**Part C: Real-World Applications and Optimization (25 marks)**

The code for the Part C was done in VScode due to the command line interface question.Added the py file with txt files of result.